In [ ]:
from keras.utils import Sequence
from keras.models import Sequential
from keras.layers import TimeDistributed, Reshape,Permute,MaxPooling2D, LSTM,Dense,Lambda, Activation, BatchNormalization, Bidirectional, GRU, LeakyReLU, Dropout, Conv2D, Flatten, Input
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras import optimizers
import numpy as np 
import cv2 
import os
import random

In [ ]:
'''
     PUT ME OUTSIDE THE TRAIN/TEST FOLDERS WITHIN THE DATASET FOLDER.
'''


In [ ]:
# #make the directories to store the resized images...
# #DO NOT RUN ME AFTER THE FOLDERS ARE POPULATED!
# os.makedirs("./train/positive/resized")
# os.makedirs("./train/negative/resized")

# os.makedirs("./test/positive/resized")
# os.makedirs("./test/negative/resized")


In [ ]:
# #run for positive, negative in train and test.
# dir = "./test/negative/"
# for root,dirs,files in os.walk(dir): #code to walk the directory
#     for x,file in enumerate(files): 
#         curdir =  os.path.join(root,file) #current directory
#         img = cv2.imread(curdir) #read in the image we are currently pointing at 
#         img = cv2.resize(img, (299,299)) #resize that image down to a size we can feed into the network
#         cv2.imwrite(dir+"resized/%d.jpg" % x, img) #write our new resized image to the resized directory w/name as num

In [2]:
#if you think you can fit the images in ram, make this a 1...
'''this flag just means whether or not the training images
   will reside in memory or not, otherwise they're read off the
   disk in real time..'''

inmemory =0 #just kidding, it's not entirely clear if this is sooo much faster, not worth it .
if inmemory:
    potholes = [cv2.imread('./train/positive/resized/%d.jpg' % x,1) for x in range(1661)]
    not_potholes = [cv2.imread('./train/negative/resized/%d.jpg' % x,1) for x in range(2365)]
    

In [3]:
'''
    note the sizes of the images are all 299,299, and then the 3 indicates an R, G, and B channel on each image.
'''
if inmemory:
    potholes = np.asarray(potholes)
    not_potholes = np.asarray(not_potholes)
    print (potholes.shape)
    print (not_potholes.shape) 

In [4]:
#datagenerator compliant with the KERAS api 
trainlen_norm = list(range(1660))
trainlen_not = list(range(2364))
class DataGenTRAIN(Sequence):
        def __init__(self, tot_ex, batch_size = 32):
                self.batch_size = batch_size
                self.tot_ex = tot_ex
                self.on_epoch_end()
                self.lwr = 0
                self.upr = batch_size
                self.half = batch_size
                list_ = list(range(0,self.tot_ex))
                np.random.shuffle(list_)
                self.master = list_
        def __getitem__(self, index):
                indices = self.master[self.lwr:self.upr]
                (X,y) =  self.__data_generation(indices = indices )
                self.lwr += self.half
                self.upr +=self.half
                return (X,y)

        def __len__(self):
                return int(np.floor((self.tot_ex) / self.batch_size))
        def on_epoch_end(self):
                self.lwr = 0
                self.upr = self.batch_size
                return

        def __data_generation(self, indices):
                #X, X_1 are both good and bad images, then they get shuffled along with their indices
                normidxs = random.sample(trainlen_norm, self.batch_size//2) 
                notidxs = random.sample(trainlen_not, self.batch_size//2)
                if inmemory:
                    X= np.array([potholes[x] for x in normidxs])
                else:
                    X= np.array([cv2.imread('./train/positive/resized/%d.jpg' % x,1) for x in normidxs])
                Y = np.ones(self.batch_size//2)
                
                if inmemory:
                    X_1= np.array([not_potholes[x] for x in normidxs])
                else:
                    X_1= np.array([cv2.imread('./train/negative/resized/%d.jpg' % x,1) for x in notidxs])
                Y_1 = np.zeros(self.batch_size//2) 
                indices = list(range(self.batch_size))
                np.random.shuffle(indices)
                X = np.append(X,X_1, axis =0)
                Y = np.append(Y,Y_1,axis=0)
                #print(len(X))
                #print(len(Y))
                X = X[indices] 
                Y = Y[indices]                
                return (X,Y)

In [5]:
#we will use InceptionV3 with a custom top and train...
#inception is a stock network that we will top off, these are famous nets that have won the 'ImageNet' competition
inception = InceptionV3(include_top=False, weights='imagenet', classes = 1)
input = Input(shape = (299,299,3))
x = inception(input)
x = Flatten()(x) 
x = Dense(128, activation = 'relu')(x) 
x = Dense(16, activation = 'relu')(x)
output = Dense(1, activation = 'sigmoid')(x)

In [6]:
model = Model(input = input, output = output)
sgd = optimizers.SGD(lr=0.001, momentum=0.9, decay=1e-6, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd)

traingenerator = DataGenTRAIN(batch_size = 8, tot_ex =1630)

model.fit_generator(generator=traingenerator,workers=1, epochs = 3)


C:\Users\Erik\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


Epoch 1/3
203/203 [==============================] - 37s 182ms/step - loss: 0.2296
Epoch 2/3
203/203 [==============================] - 24s 120ms/step - loss: 0.0785
Epoch 3/3
203/203 [==============================] - 24s 119ms/step - loss: 0.0592


In [ ]:
'''
    MAKE TESTING CODE 
    CHECK FOR BUGS, CONFIRM IN INMEMORY IS ACTUALLY FASTER OR NOT 
    ~~~

'''